In [1]:
!pip install pymysql

In [2]:
import pymysql

In [3]:
connection = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = 'admin',
    database = 'accidents')

In [4]:
cursor = connection.cursor()

In [5]:
cursor.execute("select vehicle_type from vehicle_types where vehicle_type like '%torcycle%';")

6

In [6]:
cycle_list = cursor.fetchall()
print(cycle_list)

(('Motorcycle 50cc and under\r',), ('Motorcycle 125cc and under\r',), ('Motorcycle over 125cc and up to 500cc\r',), ('Motorcycle over 500cc\r',), ('Electric motorcycle\r',), ('Motorcycle - unknown cc\r',))


In [7]:
selectSQL = ('''select vt.vehicle_type, a.accident_severity
             from accident a 
             join vehicles v on a.accident_index = v.accident_index 
             join vehicle_types vt on v.vehicle_type = vt.vehicle_code
             where vt.vehicle_type like %s
             order by a.accident_severity;
             ''')


In [8]:
insert_SQL = ('''insert into accidents_median
                values(%s,%s);''')

In [9]:
for cycle in cycle_list:
    cursor.execute(selectSQL, cycle[0])
    accidents = cursor.fetchall()
    
    quotient, remainder = divmod(len(accidents),2)
    
    if remainder:
        median_severity = accidents[quotient][1]
    else:
        median_severity = (accidents[quotient][1] + accidents[quotient+1][1])/2
    
    print("Finding Median Severity for ",cycle[0])
    
    cursor.execute(insert_SQL, (cycle[0],median_severity))

Finding Median Severity for  Motorcycle 50cc and under
Finding Median Severity for  Motorcycle 125cc and under
Finding Median Severity for  Motorcycle over 125cc and up to 500cc
Finding Median Severity for  Motorcycle over 500cc
Finding Median Severity for  Electric motorcycle
Finding Median Severity for  Motorcycle - unknown cc


In [10]:
connection.commit()
connection.close()